Written by Chris Puzzo using code sourced from maximelabonne and Suman Das

5/8/2024

Note: THIS HAS TO RUN ON GPU.

Run this to make sure you have all needed packages are installed.

In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 45.4 MB/s eta 0:00:00


Next import all the modules needed. This code has you login with huggingface credentials. The model you are using is saved on my personal account, I believe it is accessible publically.

In [2]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig,
    pipeline
)

import torch
import time
import pandas as pd
import numpy as np

from huggingface_hub import interpreter_login
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): ··········
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


This is to make sure the model is loaded in with the correct bits. It was trained using 4_bit so we want to make sure it is initialized here with 4.

In [3]:
# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)


bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)


Tell the model what GPU to to use and the model to load.

In [4]:
# Load the entire model on the GPU 0
device_map = {"": 0}

model_name = "ChrisPuzzo/llama-2-7b-privacy"



Inililizes the model to be used for prompting. Takes like 12 minutues to load.

In [5]:
# Use a pipeline as a high-level helper
from transformers import pipeline
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
tokenizer = AutoTokenizer.from_pretrained("ChrisPuzzo/llama-2-7b-privacy")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

This code creates the pipeline for text geberation. It limits the max token
length to 4096 tokens (Llama's max prefered tokens).

In [18]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=4096)

This is an example prompt

In [7]:
prompt = "Tell me a little bit about how companies use my data"

resultList is used to host the responses for future evaluation.

In [10]:
resultList = []

In [22]:
result = pipe(f"<s>[INST] {prompt} [/INST]")
print('\n',result[0]['generated_text'])
resultList.append(result[0]['generated_text'])

Input length of input_ids is 4451, but `max_length` is set to 4069. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.36 GiB. GPU 0 has a total capacity of 14.75 GiB of which 755.06 MiB is free. Process 1950 has 14.01 GiB memory in use. Of the allocated memory 11.65 GiB is allocated by PyTorch, and 2.23 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

Run one of these boxes to test the prompt in it. To create your own prompt use the following syntax

~~~
prompt = "your question in this policy:" + policy
~~~


Use the policies at the bottom for the policy variable. The policy variable at the bottom of the page has to be set first.

In [14]:
prompt = "what types of personal data are collected according to this policy: " + policy

In [ ]:
prompt = "Does this policy address data sharing with third parties: " + policy

In [ ]:
prompt = "How does this company share user data with third parties according to this policy" + policy

In [ ]:
prompt = "How does this company collect data according to this policy: " + policy

In [ ]:
prompt = "How can I opt out of data collection or marketing according to this policy: " + policy

In [ ]:
prompt = "How often is this policy reviewed and updated: " + policy

In [ ]:
prompt = "How does this policy cover the use of cookies: " + policy

In [ ]:
prompt = "Does this policy describe the use of third party services: " + policy

In [ ]:
prompt = "How does this policy govern the use of my data: " + policy

In [ ]:
prompt = "Summerize this policy for me: " + policy

In [ ]:
prompt = "How can I opt out of data collection or marketing communications from this policy: " + policy

In [ ]:
prompt = "How can this policy be improved: " + policy

In [20]:
topHive = """
TopHive website is owned by TopHive, a data controller of your personal data.

We have adopted this Privacy Policy, which determines how we are processing the information collected by TopHive, and also provides the reasons why we must collect certain personal data of you. Therefore, you must read this Privacy Policy before using TopHive website.
We take care of your personal data and undertake to guarantee its confidentiality and security.
PERSONAL INFORMATION WE COLLECT:

When you visit TopHive, we automatically collect certain information about your device, including your web browser, IP address, time zone, and some of the installed cookies on your device. Additionally, as you browse the Site, we collect information about the individual web pages or products you view, what websites or search terms referred you to the Site, and how you interact with the Site. We refer to this automatically collected information as “Device Information.” Moreover, we might collect the personal data you provide to us (including, but not limited to, Name, Address, Company, Email Address etc.) during registration to fulfill the agreement.

Why do we process your data?

Our top priority is customer data security and as such, we may process only minimal user data, only as much as it is absolutely necessary to maintain the website. Information collected automatically is used only to identify users for marketing activities.

You can visit the website without telling us who you are or revealing any information by which someone could identify you as a specific, identifiable individual. If, however, you wish to use some of the website’s features, or you wish to receive our newsletter or provide other details by filling out a form, you may provide personal data to us, such as your email, name, last name, city of residence, organization, telephone number. You can choose not to provide us with your personal data, but then you may not be able to take advantage of some of the website’s features. For example, you won’t be able to receive our Newsletter or contact us directly from the website. Users who are uncertain about what information is mandatory are welcome to contact us via privacy@tophive.ai
"""

starbucks = """

Starbucks Privacy Notice

Last Revised: March 29, 2024

Overview

This Starbucks Privacy Notice describes the types of personal information that Starbucks Corporation and its respective subsidiaries and affiliated companies (“Starbucks,” “we,” and “us”) collect, how we use it, how and when it may be shared, and the rights and choices you have with respect to your information. It also explains how we communicate with you and how you can make requests or submit inquiries to us about your information. Our goal is to help you understand how we use your information to improve our products, services, marketing, and interactions with you, as part of our commitment to maintaining your trust. Thank you for taking the time to read and understand our data and privacy related practices.

By continuing to use Starbucks Services you acknowledge this Privacy Notice and agree to our Terms of Use and other important policies, all available on our Online Policies page.


1. Applicability and Scope

This Privacy Notice (“Notice”) applies to the websites located at starbucks.com, starbucksreserve.com, princi.com, starbucks.ca, the Starbucks mobile applications available at https://www.starbucks.com/coffeehouse/mobile-apps (“Starbucks App”), when you visit our stores, and through any other websites or applications owned and operated by Starbucks brands or products that direct the viewer or user to this Notice (“Starbucks Services”). Our websites and mobile applications may offer links to websites and/or other mobile applications that are operated by third parties and not by Starbucks. If you visit one of these linked websites, you should read the website’s privacy notice, terms and conditions, and their other policies. We are not responsible for the policies and practices of third parties. Any information you give to those organizations is governed under their privacy notice, terms and conditions, and other policies.

2. Updates to this Privacy Notice

We may update this Notice from time to time. We will notify you of material changes to this Notice and will update the Last Revised date on this Notice. We encourage you to look for updates and changes to this Notice when you access our websites and mobile applications.

3. Information We Collect

When you visit and use the Starbucks Services, we may collect the following categories of information:

Identifiers – We may collect information that can be used to identify you, such as your first and last name, phone number, your address book, username and password, email address, postal address, IP address, day and month of your birthday, and demographic information (such as gender).
Financial Information – We may also collect information such as your credit card and debit card information, telephone number, and name.
Commercial Information – We may collect information about your transactions in our stores, on our websites or via our Starbucks App including what products you purchase, how frequently you purchase them, any Rewards or promotions associated with a purchase, and the products you have placed on your “Wishlist” or “My Bag” for future purchase.
Electronic Network Activity Information – We may collect information related to your electronic network activity, such as your browsing history, information about how you use the Starbucks Services, information about your device, and information related to Starbucks website’s “live help” chat feature.
Geolocation Information – We may collect information about your location of your device, such as information derived from your device (e.g., based on a browser or device’s IP address).  We may also use location-based technology in our retail locations, to collect information about the presence of your device, if your Bluetooth is turned on and your device settings allow for this.
Audio and Visual Information – We may collect audio and visual information when you use the Starbucks Services.  For example, closed circuit video surveillance cameras (“CCTV”) may be installed at some of our retail locations to monitor building security and assist in crime prevention, detection, and investigation, and to ensure the safety of our staff and visitors to our facilities, and these CCTV cameras may collect audio, visual, and similar information.  Also, depending on your choices, we may collect audio information when you use voice ordering.
Inferences – We may also collect inferences drawn from the other information described above.
Other – We may collect other information that you voluntarily provide when you communicate with us.
Additionally, depending on how you use the Starbucks Services, we may collect the following categories of sensitive personal information from you:

Precise Geolocation Information – We may collect information about the precise location of your device (e.g., GPS latitude and longitude), consistent with your device settings. We may also use location-based technology in our retail locations, to collect information about the presence of your device, if your Bluetooth is turned on and your device settings allow for this. We may collect this location information or derive it from your GPS, WiFi, Bluetooth or other location-based technology for purposes of facilitating our services, such as to provide you with information about stores near you, to enable you to remotely order and pay for our products and services, or so that you may have certain Starbucks products delivered to you by a third-party vendor.
We may aggregate or de-identify the information described above.  Aggregated or de-identified data that we do not attempt to reidentify is not subject to this Notice.

Without this information, we may not be able to provide you with all the requested services.

4. Sources of Information
In addition to the information we receive from you, we may also receive the categories of information described above from other sources including from other users of Starbucks Services (such as when they purchase a gift for you), advertising partners, co-sponsors, internet service providers, data analytics providers, operating systems and platforms, social networks, and publicly available sources.  For example, if you access any social media or similar services through the Starbucks Services to login or to share information about your experience on our Starbucks Services with others, we may collect information from these third-party services.

5. How We Use Your Information

We may use your information for business purposes, including to provide the products and services you request, to perform customer service functions, for security and fraud prevention, for marketing and promotional purposes, and to perform website and mobile application analytics.  We may use the information we collect about you to:

(a)  To Provide Our Services – We may process your information when you access or use the Starbucks Services, including to:

process or fulfill orders and transactions;
maintain and service accounts;
support and improve our loyalty programs, such as Starbucks® Rewards;
deliver gift cards or e-gifts, and any associated personalized messages, in accordance with your instructions;
debug to identify and repair errors that impair existing functionality;
facilitate the functionality of our websites and mobile applications;
process payments; and
customize experiences and personalization when you are in and around our stores, if you have given permission for location sharing.
(b)  To Communicate With You – We may process certain information in order to communicate with you in relation to your accounts, our services, our marketing, and your requests, including to:

provide customer service, including when you interact with our customer service agents through our website’s “live help” chat feature, and requests for information;
communicate with you about orders, purchases, returns, services, accounts, programs, contests, and sweepstakes;
send you personalized promotions, content, and special offers;
communicate with you about our brands, products, events, or other promotional purposes;
implement your communications preferences, such as sharing information with Starbucks business partners so that they may email you about their promotions, products and initiatives; and
provide important product safety information and notice of product recalls.
(c)  For Research, Development, and Improvement of Our Services – We want to ensure that our website, mobile applications, and services are continually improving and expanding so that we meet and exceed your needs and expectations. To do so, we may process certain personal information, including to:

maintain, improve, and analyze our websites, mobile applications, ads, and the products and services we offer; and
help to ensure security and integrity of the Starbucks Services, including to detect, prevent, or investigate suspicious activity or fraud.
(d)  To Enforce our Terms, Agreements, or Policies – To maintain a safe, secure, and trusted environment for you when you use our websites, mobile applications, and other services, we may use your personal information to ensure our terms, policies, and agreements with you and any third parties are enforced.

(e)  To Comply with Applicable Laws – We may be required to process certain information under certain laws and regulations, such as tax laws, as well as to:

maintain appropriate records for internal administrative purposes; and
comply with applicable legal and regulatory obligations, and respond to lawful governmental requests, as needed.
(f)  Targeted Advertising – We may process your information to provide you with targeted advertising (also known as cross-contextual advertising and online behavioral advertising) and offers and promotions that you may be interested in.

(g)  With Your Consent – We may process certain information in order to fulfill any other business or commercial purposes at your direction or with your consent.

6. How We Disclose Your Information

We may disclose your information as needed to fulfill the purposes described in this Notice and as permitted by applicable law.

 (a) When We Work Together – We may share information between and among Starbucks Corporation, its subsidiaries, and affiliated companies for purposes of management and analysis, decision making, and other business purposes. For example, we may share your information with our subsidiaries and affiliated companies to administer our loyalty programs, process orders and requests, and expand and promote our product and service offerings.

(b)  When We Work with Service Providers & Business Partners – We may share identifiers, commercial information, electronic and network activity, geolocation and precise geolocation information, audio and visual information, and inferences, as described above, for a variety of business purposes, such as payment processing, website and data hosting, location mapping, product and service delivery, customer service, advertising and marketing (including counting ad impressions, ensuring compliance with industry standards, personalization, analytics services, security and performance monitoring, maintaining and servicing accounts, processing or fulfilling orders and transactions, verifying customer information, research, auditing, and data processing. We strive to contractually prohibit these service providers and business partners from retaining, using, or disclosing your confidential personal information for any purpose other than performing agreed upon services for us.

(c)  When We Work on Business Transactions – If we become involved with a merger, corporate transaction or another situation involving the transfer of some or all of our business assets, we may share your information with business entities or people involved in the negotiation or transfer.

(d)  When Sharing Helps Us Protect Safety and Lawful Interests – We may disclose your information if we believe that the disclosure is required by law, if we believe that the disclosure is necessary to enforce our agreements or policies, or if we believe that the disclosure will help us protect the rights, property, safety of Starbucks or our customers or partners.

(e)  When You Give Consent – We may share information about you with other companies if you give us permission or direct us to share the information.

(f)  When You Post on Our Websites – If you post information on a blog or another part of our websites, the information that you post may be seen by other visitors to our websites.  We are not responsible for the information you choose to submit in these public areas.

7. Cookies, Web Beacons and Similar Technologies

We and others may use a variety of technologies to collect information about your device and use of our websites and mobile applications. These technologies include first- and third-party cookies, web beacons, JavaScript, entity tags, and HTML local storage. Most web browsers can be programmed to accept or reject the use of some or all of these technologies, although you must take additional steps to disable or control other technologies. For more information on exercising your preferences in relation to cookies and similar technologies, please see the section of this Notice titled “Your Choices & Rights”.

Cookies – Cookies are small data files that are sent from a website’s server and are stored on your device’s hard drive either for only the duration of your visit (“session cookies”) or for a fixed period (“persistent cookies”). Cookies contain information that can later be read by a web server.
You may be able to reject cookies, web beacons, entity tags and HTML5 local storage by adjusting the appropriate settings in your browser. Each browser is different, but many browsers have preferences or options that may be adjusted to allow you to either accept or reject cookies and certain other technologies before they are set or installed, or allow you to remove or reject the use or installation of certain technologies altogether. If you want to learn the correct way to modify your browser settings, please use the Help menu in your browser or review the instructions provided by the following browsers: Internet Explorer, Google Chrome, Mozilla Firefox, Safari Desktop, Safari Mobile; and Android browser.

Interest-Based Ads – Many advertising companies that collect information for interest-based advertising are members of the Digital Advertising Alliance (DAA) or the Network Advertising Initiative (NAI), both of which maintain websites where people can opt out of interest-based advertising from their members. To opt-out of website interest-based advertising provided by each organization’s respective participating companies, visit the DAA’s opt-out portal available at http://optout.aboutads.info/, the DAA of Canada’s opt-out portal available at https://youradchoices.ca/en/tools, or visit the NAI’s opt-out portal available at http://optout.networkadvertising.org/?c=1.
Residents of the European Union may opt-out of online behavioral advertising served by the European Interactive Digital Advertising Alliance’s participating member organizations by visiting https://www.youronlinechoices.eu/.

To opt-out of data collection for interest-based advertising across mobile applications by participating companies, download the DAA’s App Choices mobile application opt-out offering at https://youradchoices.com/appchoices.

Adobe Flash Player Technology – We allow Adobe to set and enable special cookies that are necessary to deliver video content for Adobe Flash Player. You cannot remove Flash cookies simply by changing your browser settings. If you would like to limit the websites that can store information in Flash cookies on your device, you must visit the Adobe website:
http://www.macromedia.com/support/documentation/en/flashplayer/help/settings_manager07.html
Web Beacons – Web beacons are small, transparent images that are embedded in web pages, applications, and emails that are sometimes called “clear gifs,” “single pixel gifs”, “page tags” or “web bugs.” We use web beacons to track the web pages you visit, to test the effectiveness of our marketing, and to find out if an email has been opened and acted on.
JavaScript – JavaScript are code snippets embedded in various parts of websites and applications that facilitate a variety of operations including accelerating the refresh speed of certain functionality or monitoring usage of various online components.
Entity Tags -- Entity Tags are HTTP code mechanisms that allow portions of websites to be stored or “cached” within your browser and validates these caches when the website is opened, accelerating website performance since the web server does not need to send a full response if the content has not changed.
HTML5 Local Storage – HTML5 local storage allows data from websites to be stored or “cached” within your browser to store and retrieve data in HTML5 pages when the website is revisited. These and similar technologies are used for the following purposes:
Services and Functionality – Some of these technologies are required to allow you to access and use our websites or mobile applications and the various services and functionality we offer. Without these technologies, some of the services and functionality on our websites and mobile applications would not work properly.
Performance Monitoring – Some of these technologies help us analyze and estimate traffic and assess the volume of use of services and functionality on websites and mobile applications. They show us how visitors and customers interact with our digital properties, whether there are any errors, the degree to which certain web pages, applications, services or functionality are accessed and used and how they are performing or operating. When these technologies are used for performance monitoring, no directly identifiable information (e.g., your name, email address or phone number) is collected.
User Convenience – Some of these technologies enhance the ease of use of our websites and mobile applications and the services and functionality they make available by accelerating load and refresh times and remembering information that you have provided on prior visits to our websites or when you last used a website or mobile application service or functionality.
Marketing – Some of these technologies are used to tailor your experience on our website, within our mobile applications, or across your applications or devices by controlling the promotions, advertisements and other marketing messages that may appear when you access or use these or other digital properties.
Taken together, these technologies help us learn which services and functionality you are using and how you are accessing information about us. We may use this information to personalize your visit to a website or use of a mobile application or to send you relevant promotions and advertisements.

8. How We Protect Your Information

Starbucks uses reasonable technical, physical, and administrative security measures to reduce the risk of loss, misuse, unauthorized access, disclosure or modification of your information. However, no security system is perfect, and due to the inherent nature of the Internet, we cannot guarantee that data, including personal information, is absolutely safe from intrusion or other unauthorized access by others. You are responsible for protecting your password(s) and maintaining the security of your devices.

9. Retention

Starbucks stores information as reasonably necessary and proportionate to accomplish the purposes identified in this Notice based on criteria such as the length of time we need to provide the services to you, and to meet legal requirements, including record retention, resolving disputes, and enforcing our agreements. Our retention of your information is governed by applicable law.

The personal information that you provide us is stored and processed on servers owned by Starbucks and other companies in the United States, Canada or elsewhere around the world, including locations outside of the country or jurisdiction where you are located. Regardless of the company or person who processes your information and where it is processed, we will take steps to transfer and protect your information through appropriate safeguards in accordance with applicable data protection laws and this Notice.

"""




Set Policy equal to one of the ones above

In [21]:
policy = starbucks